*`Modelo de aprendizaje automático`**: 

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un **sistema de recomendación**. Para ello, te ofrecen dos propuestas de trabajo: En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la *similitud del coseno*. 
La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”. 
Deben crear al menos **uno** de los dos sistemas de recomendación (Si se atreven a tomar el desafío, para mostrar su capacidad al equipo, ¡pueden hacer ambos!). Tu líder pide que el modelo derive obligatoriamente en un GET/POST en la API símil al siguiente formato:

Si es un sistema de recomendación item-item:
+ def **recomendacion_juego( *`id de producto`* )**:
    Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

Si es un sistema de recomendación user-item:
+ def **recomendacion_usuario( *`id de usuario`* )**:
    Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [3]:
# Importando librerias
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
df_games = pd.read_parquet('dfgamesrecomendacionAPI.parquet', engine='pyarrow')
df_games.head(2)

,item_id,app_name,title,genres,release_date,developer,price
24777,384550,Painters Guild,Painters Guild,Management,2015,Lucas Molina,9.99
30485,420440,Atlantic Fleet,Atlantic Fleet,World War II,2016,Killerfish Games,9.99


In [4]:
#inicio un archivo con las columnas que tengo que reemplazar nulos
archivo_columnas=["app_name","developer","genres",'title']

#genero un dataframe con las columnas y sus nulos reemplazados por "Sin Datos"
nulos= df_games[archivo_columnas].fillna("Sin Datos")

#elimino las colummas a reemplazar y concateno ambos dataframes 
games_2 = pd.concat([df_games.drop(archivo_columnas, axis=1), nulos], axis=1)

In [5]:
modelo_item= games_2[["item_id", "app_name", "genres"]]  #SOLO TOMO LAS VARIABLES DE MI INTERES.
# EL ITEM ID QUE ES EL ID DE PRODUCTO,  el app y genres

In [8]:
modelo_item_csv = "Modeloaprendizaje.CSV"
modelo_item.to_csv(modelo_item_csv , index=False, encoding="utf-8")

In [9]:
# Paso 1: Contar la frecuencia de cada género
genre_counts = modelo_item['genres'].value_counts()

# Paso 2: Seleccionar los 20 géneros más frecuentes
top_20_genres = genre_counts.head(20).index

# Paso 3: Filtrar los datos para incluir solo estos 20 géneros
filtered_modelo_item = modelo_item[modelo_item['genres'].isin(top_20_genres)]

# Paso 4: Convertir estos géneros seleccionados en variables dummy
modelo_item_2 = pd.get_dummies(filtered_modelo_item, columns=['genres'], prefix='', prefix_sep='')
# Mostrar el resultado
print(modelo_item_2.head())

        item_id                                  app_name     2D  Action  \
102783   463100                Liveza: Death of the Earth  False   False   
74512    506500                               Party Panic  False    True   
121124   366610                     Not without my donuts  False   False   
19863    335300  DARK SOULS™ II: Scholar of the First Sin  False    True   
70447    712380                             Temple Escape  False   False   

        Adventure  Atmospheric  Casual  Co-op  Early Access  Fantasy  ...  \
102783      False        False   False  False         False    False  ...   
74512       False        False   False  False         False    False  ...   
121124       True        False   False  False         False    False  ...   
19863       False        False   False  False         False    False  ...   
70447       False        False    True  False         False    False  ...   

        Indie  Multiplayer  Platformer  Puzzle    RPG  Sci-fi  Simulation  \
102

In [10]:
modelo_item_2.head(3)

,item_id,app_name,2D,Action,Adventure,Atmospheric,Casual,Co-op,Early Access,Fantasy,...,Indie,Multiplayer,Platformer,Puzzle,RPG,Sci-fi,Simulation,Singleplayer,Sports,Strategy
102783,463100,Liveza: Death of the Earth,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
74512,506500,Party Panic,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
121124,366610,Not without my donuts,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
modelo_item_3= modelo_item_2.groupby(["item_id","app_name"]).sum().reset_index()
modelo_item_3.head(4)

,item_id,app_name,2D,Action,Adventure,Atmospheric,Casual,Co-op,Early Access,Fantasy,...,Indie,Multiplayer,Platformer,Puzzle,RPG,Sci-fi,Simulation,Singleplayer,Sports,Strategy
0,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40,Deathmatch Classic,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,60,Ricochet,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
modelo_item_3['item_id']=modelo_item_3['item_id'].astype('string')


In [13]:
modelo_item_3['item_id']=modelo_item_3['item_id'].astype('int')

In [14]:
# verificar todas las variables
modelo_item_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7415 entries, 0 to 7414
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           7415 non-null   int32 
 1   app_name          7415 non-null   object
 2   2D                7415 non-null   int64 
 3   Action            7415 non-null   int64 
 4   Adventure         7415 non-null   int64 
 5   Atmospheric       7415 non-null   int64 
 6   Casual            7415 non-null   int64 
 7   Co-op             7415 non-null   int64 
 8   Early Access      7415 non-null   int64 
 9   Fantasy           7415 non-null   int64 
 10  Free to Play      7415 non-null   int64 
 11  Great Soundtrack  7415 non-null   int64 
 12  Indie             7415 non-null   int64 
 13  Multiplayer       7415 non-null   int64 
 14  Platformer        7415 non-null   int64 
 15  Puzzle            7415 non-null   int64 
 16  RPG               7415 non-null   int64 
 17  Sci-fi        

In [15]:
# y aqui aplicamos el modelos que nos recomendaron, que es el modelo coseno
similitudes = cosine_similarity(modelo_item_3.iloc[:,3:])

In [16]:
similitudes

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.]])

In [17]:
# como tardo mucho vamos a hacer mas pequeños los datos, ya que si no la API NO VA A FUNCIONAR
modelo_final_csv = "modelo_final.csv" # esta tabla excede la memoria de la api
modelo_item_3.to_csv(modelo_final_csv , index=False, encoding="utf-8")

In [18]:
cant_filas= len(modelo_item_3)      # hacemos mas pequeños los datos
#Calculo la mitad
mitad_filas= cant_filas // 20
#Selecciono la mitad superior
modelo_render= modelo_item_3.iloc[:mitad_filas]

In [19]:
modelo_render.shape
modelo_render.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           370 non-null    int32 
 1   app_name          370 non-null    object
 2   2D                370 non-null    int64 
 3   Action            370 non-null    int64 
 4   Adventure         370 non-null    int64 
 5   Atmospheric       370 non-null    int64 
 6   Casual            370 non-null    int64 
 7   Co-op             370 non-null    int64 
 8   Early Access      370 non-null    int64 
 9   Fantasy           370 non-null    int64 
 10  Free to Play      370 non-null    int64 
 11  Great Soundtrack  370 non-null    int64 
 12  Indie             370 non-null    int64 
 13  Multiplayer       370 non-null    int64 
 14  Platformer        370 non-null    int64 
 15  Puzzle            370 non-null    int64 
 16  RPG               370 non-null    int64 
 17  Sci-fi          

In [20]:
similitudes_render = cosine_similarity(modelo_render.iloc[:,3:])

In [21]:
modelo_render.head()

,item_id,app_name,2D,Action,Adventure,Atmospheric,Casual,Co-op,Early Access,Fantasy,...,Indie,Multiplayer,Platformer,Puzzle,RPG,Sci-fi,Simulation,Singleplayer,Sports,Strategy
0,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40,Deathmatch Classic,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,60,Ricochet,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,70,Half-Life,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [22]:
modelo_render.to_parquet("modelo_render.parquet")

In [23]:
modelo_render

,item_id,app_name,2D,Action,Adventure,Atmospheric,Casual,Co-op,Early Access,Fantasy,...,Indie,Multiplayer,Platformer,Puzzle,RPG,Sci-fi,Simulation,Singleplayer,Sports,Strategy
0,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40,Deathmatch Classic,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,60,Ricochet,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,70,Half-Life,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,55020,Air Forte,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
366,55040,Atom Zombie Smasher,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
367,55110,Red Faction®: Armageddon™,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
368,55140,MX vs. ATV Reflex,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [4]:

ruta_archivo = 'modelo_render.parquet'
modelo_render = pd.read_parquet(ruta_archivo, engine='pyarrow')
def recomendacion_juego(item_id): #este si funciono tal cual
    
    game = modelo_render[modelo_render['item_id'] == item_id]
    
    if game.empty:
        return("El juego '{item_id}' no posee registros.")
    
    # Obtiene el índice del juego dado
    idx = game.index[0]

    # Toma una muestra aleatoria del DataFrame df_games
    sample_size = 100  # Define el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_render.sample(n=sample_size, random_state=42)  # Ajusta la semilla aleatoria según sea necesario

    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_render.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [5]:
# ver las recomendaciones segun el item_id ejemplo
recomendacion_juego(2360)   # el juego es 204 counter strike el cual es un shooter de accion y las recomendaciones son parecidas.

{'similar_games': ['Golden Axe™',
  'X-Blades',
  'King Arthur II: The Role-Playing Wargame',
  'DeathSpank',
  'Wolfenstein 3D']}

In [6]:
recomendacion_juego(20)  # el juego 35140 es batman y te recomienda otros videojuegos sobre peliculas y accion 

{'similar_games': ['Crysis Warhead®',
  'Lara Croft GoL: Hazardous Reunion - Challenge Pack 3',
  'Sam & Max 106: Bright Side of the Moon',
  'Just Cause 2 - Black Market Boom Pack DLC',
  'Sam & Max 102: Situation: Comedy']}